<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Model IO Exercise 

The purpose of this exercise is to test your understanding of building out Model IO systems. You will also hopefully notice the need to chain responses together, which we will cover later in this course!

Watch the video for a full overview on minimum outputs this class should be capable of, but feel free to expand on this project, or to just treat it as a code-along!

## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

### Suggested Imports

Feel free to accomplish this task however you prefer!

In [109]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import date
from langchain_ollama.chat_models import ChatOllama
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain_core.caches import InMemoryCache

In [128]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        systemMessage = SystemMessagePromptTemplate.from_template(template="Your are History teacher who want to check the knowledge of the students using the questions, Only ask question about historical dates. just ask a question directly,")
        HumanMessage = HumanMessagePromptTemplate.from_template(template="Ask me the history question on the {topic}")

        template = ChatPromptTemplate.from_messages([systemMessage, HumanMessage])
        cache = InMemoryCache()
        chat = ChatOllama(
            model = "llama3.2:1b",
            temperature = 0.8,
            num_predict = 200,
            cache = cache
        )
        question = chat.invoke(template.format(topic = topic)).content
        return question
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        dateParser = DatetimeOutputParser()
        systemMessage = SystemMessagePromptTemplate.from_template(template = "Give straight forward answer in date to the given question")
        humanMessage = HumanMessagePromptTemplate.from_template(template= 'What year did World War II begin?')
        AIMessage = AIMessagePromptTemplate.from_template(template = '01-07-1939')
        humanMessage1 = HumanMessagePromptTemplate.from_template(template="{question}")
        template = ChatPromptTemplate.from_messages([systemMessage, humanMessage, AIMessage, humanMessage1])
        cache = InMemoryCache()

        chat = ChatOllama(
            model = "llama3.2:1b",
            temperature = 0.2,
            num_predict = 200,
            cache = cache

        )
        correct_datetime = chat.invoke(template.format(question = question)).content
        
        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(question)
        year = int(input("Year : "))
        month = int(input("Month : "))
        day = int(input("Day : "))
        user_datetime = date(year, month, day)
        return user_datetime
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        systemMessage = SystemMessagePromptTemplate.from_template(template = "You will be provided with two dates, just say whether dates matches or not, and if not then the difference between those dates as output")
        humanMessage = HumanMessagePromptTemplate.from_template(template="What is the difference between these two dates?\n{user_answer} and {ai_answer}")
        template = ChatPromptTemplate.from_messages([systemMessage, humanMessage])
        cache = InMemoryCache()

        chat = ChatOllama(
            model = "llama3.2:1b",
            temperature = 0.8,
            num_predict = 200,
            cache = cache

        )
        response = chat.invoke(template.format(user_answer = user_answer, ai_answer = ai_answer)).content
        return response


### Example Usage

Feel free to expand or edit this project. To keep things simple we have every method return an object that will then feed into a new method!

In [129]:
quiz_bot = HistoryQuiz()

In [130]:
question = quiz_bot.create_history_question(topic='World War 2')

In [131]:
question

"Let's start with a basic question. Which year is commonly considered the end of World War II?"

In [132]:
ai_answer = quiz_bot.get_AI_answer(question)

In [133]:
# Day After Pearl Harbor
ai_answer

'1945'

In [134]:
user_answer = quiz_bot.get_user_answer(question)

Let's start with a basic question. Which year is commonly considered the end of World War II?


In [135]:
user_answer

datetime.date(1945, 8, 1)

In [136]:
quiz_bot.check_user_answer(user_answer,ai_answer)

'1945-08-01 does not match 1945. \n1945 - 1945 = 0 days'